In [1]:
from pathlib import Path
paths = [str(x) for x in Path('/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/').glob('**/*.txt')]

In [2]:
len(paths)

158

In [3]:
paths

['/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_9100000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_5500000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_8500000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_8400000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_6600000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_7600000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_14300000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/tokenized_shard_4700000.txt',
 '/userdirs/piyumal/roberta_sinhala/content/sinhala-dat

In [5]:
from transformers import DistilBertTokenizerFast, RobertaTokenizerFast, RobertaTokenizer
tokenizer = RobertaTokenizer(vocab_file='/userdirs/piyumal/roberta_sinhala/Roberta_tokenizer_30000/vocab.json',
                             merges_file='/userdirs/piyumal/roberta_sinhala/Roberta_tokenizer_30000/merges.txt', max_len=512)

In [6]:
tokens = tokenizer.encode("ඉතිං බ්ලොග් පිටුවත් ලිව්වා නම් හොදයි කියලා හිතුනා.")

In [7]:
tokenizer.decode(tokens)

'<s>ඉතිං බ්ලොග් පිටුවත් ලිව්වා නම් හොදයි කියලා හිතුනා.</s>'

In [8]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [9]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_000,
    max_position_embeddings=514,
    num_attention_heads=6,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [10]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

# model = RobertaForMaskedLM.from_pretrained('/userdirs/piyumal/roberta_sinhala/ROberta_models_2/6_layer_model_0_6')

In [11]:
model.num_parameters()
# => 84 million parameters

66586416

In [ ]:
from transformers import TrainerCallback
class PrinterCallback(TrainerCallback):

    def on_epoch_end(self, args, state, control, logs=None, **kwargs):
        

In [12]:
from transformers import LineByLineTextDataset,TextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

epochs = 1
for epoch in range(epochs):
    for j, path in enumerate(paths):
        dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=path, block_size =512)
        data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
        training_args = TrainingArguments(
                            output_dir="./Roberta_models_tokenizer_30000",
                            overwrite_output_dir=True,
                            num_train_epochs=2,
                            per_gpu_train_batch_size=16,
                            save_steps=10_000,
                            save_total_limit=2,
                            prediction_loss_only=True,
                            logging_steps = 100)
        trainer = Trainer(
                    model=model,
                    args=training_args,
                    data_collator=data_collator,
                    train_dataset=dataset)
        trainer.train()
        trainer.save_model(f'./Roberta_models_tokenizer_30000/6_layer_model_{epoch}_{j}')
        with open('loss_value_file_{j}', 'wb') as fp:
                pickle.dump(trainer.state.log_history, fp)


        

/userdirs/piyumal/roberta_sinhala/rpt_env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 100000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12500


Step,Training Loss
2500,4.186000
5000,3.311400
7500,2.961300
10000,2.789500
12500,2.693300


Saving model checkpoint to ./Roberta_models_tokenizer_30000/checkpoint-10000
Configuration saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./Roberta_models_tokenizer_30000/6_layer_model_0_0
Configuration saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_0/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_0/pytorch_model.bin
/userdirs/piyumal/roberta_sinhala/rpt_env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_ml

Step,Training Loss
2500,2.681300
5000,2.461000
7500,2.298000
10000,2.179000
12500,2.124500


Saving model checkpoint to ./Roberta_models_tokenizer_30000/checkpoint-10000
Configuration saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./Roberta_models_tokenizer_30000/6_layer_model_0_1
Configuration saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_1/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_1/pytorch_model.bin
/userdirs/piyumal/roberta_sinhala/rpt_env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_ml

...

Saving model checkpoint to ./Roberta_models_tokenizer_30000/checkpoint-10000
Configuration saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./Roberta_models_tokenizer_30000/6_layer_model_0_155
Configuration saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_155/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_155/pytorch_model.bin
/userdirs/piyumal/roberta_sinhala/rpt_env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/

Step,Training Loss
2500,0.810400
5000,0.805400
7500,0.781300
10000,0.762800
12500,0.743000


Saving model checkpoint to ./Roberta_models_tokenizer_30000/checkpoint-10000
Configuration saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./Roberta_models_tokenizer_30000/6_layer_model_0_156
Configuration saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_156/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_156/pytorch_model.bin
/userdirs/piyumal/roberta_sinhala/rpt_env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/

Step,Training Loss
2500,0.813800
5000,0.807900
7500,0.777900
10000,0.751400
12500,0.746900


Saving model checkpoint to ./Roberta_models_tokenizer_30000/checkpoint-10000
Configuration saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./Roberta_models_tokenizer_30000/6_layer_model_0_157
Configuration saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_157/config.json
Model weights saved in ./Roberta_models_tokenizer_30000/6_layer_model_0_157/pytorch_model.bin


In [20]:
trainer.state.log_history[-1]['epoch']

2.0